In [9]:
from google_play_scraper import reviews
import pandas as pd

def scrape_reviews_for_app(app_id, bank_name, total_reviews=400):
    all_reviews = []
    count = 0
    batch_size = 100
    
    while count < total_reviews:
        try:
            r, _ = reviews(
                app_id,
                lang='en',
                country='us',
                count=batch_size,
                sort=reviews.Sort.NEWEST
            )
            for item in r:
                all_reviews.append({
                    'review': item['content'],
                    'rating': item['score'],
                    'date': item['at'].date(),
                    'bank': bank_name
                })
            count += len(r)
            print(f"Collected {count} reviews for {bank_name}")
        except Exception as e:
            print(f"Error scraping {bank_name}: {e}")
            break
    
    return pd.DataFrame(all_reviews)


In [10]:
if __name__ == "__main__":
    cbe_df = scrape_reviews_for_app("com.combanketh.mobilebanking", 'CBE')
    boa_df = scrape_reviews_for_app("com.boa.boaMobileBanking", 'BOA')
    dashen_df = scrape_reviews_for_app("com.dashen.dashensuperapp", 'Dashen')

    all_reviews_df = pd.concat([cbe_df, boa_df, dashen_df], ignore_index=True)
    all_reviews_df.to_csv("raw_reviews.csv", index=False)

Error scraping CBE: 'function' object has no attribute 'Sort'
Error scraping BOA: 'function' object has no attribute 'Sort'
Error scraping Dashen: 'function' object has no attribute 'Sort'


In [4]:
import pandas as pd

def preprocess_reviews(file_path, output_path):
    df = pd.read_csv(file_path)

    # Drop duplicates
    df.drop_duplicates(subset=['review'], inplace=True)

    # Drop rows with missing critical values
    df.dropna(subset=['review', 'rating', 'date', 'bank'], inplace=True)

    # Normalize date
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.date
    df.dropna(subset=['date'], inplace=True)

    # Add source
    df['source'] = 'Google Play Store'

    # Save cleaned CSV
    df[['review', 'rating', 'date', 'bank', 'source']].to_csv(output_path, index=False)

    print(f"Preprocessed data saved to {output_path}")
    return df


In [5]:
if __name__ == "__main__":
    preprocess_reviews("raw_reviews.csv", "cleaned_reviews.csv")

EmptyDataError: No columns to parse from file